In [155]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import nltk
import sklearn
import string
from nltk import pos_tag 
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
import re
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from nltk.stem.isri import ISRIStemmer
from sklearn.metrics import accuracy_score


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\CLS\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\CLS\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\CLS\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\CLS\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [195]:
!pip install pyarabic

In [156]:
df_train = pd.read_csv('balanced_2classes_train.csv')
df_val = pd.read_csv('balanced_2classes_val.csv')
df_test = pd.read_csv('balanced_2classes_test.csv')

In [157]:
df_train.head()

,label,user_id,book_id,text
0,0,4969402,10706934,لا جديد بالنسبة لي!
1,0,4865181,3153116,دعونا لا ننكر أن لغة علوان هنا ، جمـيلة جداً ...
2,0,8132581,3831344,الكتاب كان دون توقعاتي لعله بسبب كرهي للفلسفة...
3,0,8553746,11205152,مشدنيش في حاجة . وسلبي بشكل كبير ويميل للكآبه...
4,0,5115054,11942408,شغاال برضه بس مش واو!!


In [158]:
df_all = pd.concat([df_train,df_val,df_test],axis=0)

In [159]:
df_all.shape[0],df_train.shape[0]+df_test.shape[0]+df_val.shape[0]

(16448, 16448)

In [160]:
stop_words = pd.read_excel('ar_stops.xlsx')

In [161]:
stop_words.head()

,“
0,[
1,[
2,وأنه
3,وأني
4,“


In [162]:
stop_words.shape

(1590, 1)

In [163]:
stop_words = stop_words.values

In [164]:
stop_words.shape

(1590, 1)

In [165]:
stop_word = []

In [166]:
for item in stop_words:
    stop_word.append(item[0])

In [167]:
len(stop_word)

1590

In [168]:
# def get_wordnet_pos(pos_tag):
#     if pos_tag.startswith('J'):
#         return wordnet.ADJ
#     elif pos_tag.startswith('V'):
#         return wordnet.VERB
#     elif pos_tag.startswith('N'):
#         return wordnet.NOUN
#     elif pos_tag.startswith('R'):
#         return wordnet.ADV
#     else:
#         return wordnet.NOUN
def clean_text(text):
    # lower text
    #text = text.lower()
    # tokenize text and remove puncutation
    text = [word.strip(string.punctuation) for word in text.split(" ")]
    # remove words that contain numbers
    text = [word for word in text if not any(c.isdigit() for c in word)]
    # remove stop words
    stop = stop_word
    text = [x for x in text if x not in stop]
    # remove empty tokens
    text = [t for t in text if len(t) > 0]
    # pos tag text
    #pos_tags = pos_tag(text)
    # lemmatize text
    #text = [WordNetLemmatizer().lemmatize(t[0], get_wordnet_pos(t[1])) for t in pos_tags]
    # remove words with only one letter
    text = [ISRIStemmer().stem(t) for t in text]
    text = [t for t in text if len(t) > 1]
    # join all
    text = " ".join(text)
    sent = re.sub("^\d+\s|\s\d+\s|\s\d+$", "", text)
    sent = re.sub("[إأآا]", "ا", sent)
    sent = re.sub("[ة]", "ه", sent)
    sent = re.sub(r'(.)\1+', r'\1', sent)
    return(text)


In [ ]:
# sent = re.sub("[a-zA-z0-9]+","",sent)
#     sent = re.sub("^\d+\s|\s\d+\s|\s\d+$", "", sent)
#     sent = re.sub("[إأآا]", "ا", sent)
#     sent = re.sub("[ة]", "ه", sent)
#     sent = re.sub(r'(.)\1+', r'\1', sent)
#     sent = re.sub( r"[,!.،÷×$%^&=~|\;@#$&%*+=\-_:<>?ـ؟(){}\'\"\[\]\\]"," ",sent)
#     sent = araby.strip_tatweel(sent)
#     sent = araby.strip_harakat(sent)

In [169]:
df_all.head()

,label,user_id,book_id,text
0,0,4969402,10706934,لا جديد بالنسبة لي!
1,0,4865181,3153116,دعونا لا ننكر أن لغة علوان هنا ، جمـيلة جداً ...
2,0,8132581,3831344,الكتاب كان دون توقعاتي لعله بسبب كرهي للفلسفة...
3,0,8553746,11205152,مشدنيش في حاجة . وسلبي بشكل كبير ويميل للكآبه...
4,0,5115054,11942408,شغاال برضه بس مش واو!!


In [170]:
df_all.drop(['user_id','book_id'],inplace=True,axis=1)

In [171]:
df_all.head()

,label,text
0,0,لا جديد بالنسبة لي!
1,0,دعونا لا ننكر أن لغة علوان هنا ، جمـيلة جداً ...
2,0,الكتاب كان دون توقعاتي لعله بسبب كرهي للفلسفة...
3,0,مشدنيش في حاجة . وسلبي بشكل كبير ويميل للكآبه...
4,0,شغاال برضه بس مش واو!!


In [172]:
df_all["text"]= df_all["text"].apply(lambda x : clean_text(x))

In [173]:
df_all.head(20)

,label,text
0,0,لا جدد نسب
1,0,دعو لا نكر لغة علو جمـيل جدا لا خفى علي اني به...
2,0,كتب وقع كره لسف لعل سلب د.المسيري تقعر عالأقل ...
3,0,مشدنيش حجة سلب كبر يمل كآب كبر
4,0,شغال برض بس مش
5,1,حلى قصد
6,0,روي تحت توقع كثر بلغ عبر رائعة(صمت حنن نسب كسل...
7,1,كتب درويش بلغ قرب هذي قرب جده كثر علي علم شعر ...
8,0,كنت توقع كتب عكس نضج شخص حمد حلم زي شفن ششة شي...
9,1,طول حيت لم ختر ولم قرر انن قرر انن خار حية سأح...


In [174]:
df_train = df_all[:df_train.shape[0]]

In [175]:
df_train.head()

,label,text
0,0,لا جدد نسب
1,0,دعو لا نكر لغة علو جمـيل جدا لا خفى علي اني به...
2,0,كتب وقع كره لسف لعل سلب د.المسيري تقعر عالأقل ...
3,0,مشدنيش حجة سلب كبر يمل كآب كبر
4,0,شغال برض بس مش


In [176]:
df_val = df_all[df_train.shape[0]:df_train.shape[0]+df_val.shape[0]]

In [177]:
df_val.head()

,label,text
0,1,مية ونت ترح رحى فرق ضحى وحد عدد حجة حلة فقد سن...
1,1,شعر شدد تمز خصت شعر وطن نسى كلم مصر كلب الس بل...
2,0,كتب لخص ليبرالية عرب رأه ليبرالية عرب دعم خرج ...
3,1,حدث ثني دين عمل ثني عدل قوة بصر رمز احا وضح اح...
4,1,ديوانى فضل


In [178]:
df_test = df_all[df_val.shape[0]:df_test.shape[0]+df_val.shape[0]]

In [179]:
df_test.head()

,label,text
3288,1,حلو رار رار حلو
3289,1,لعل حدث روي أسو روت الف لغة منفلوط لمس شغف قلب...
3290,0,قرأ باب ادء وجب لا جدد سلب ولا حدث بأس لسل رجل...
3291,1,حلو ترب مره اسف ترب طلع طعم حلو بس عيز اشف سلب...
3292,0,حجة ملة ثير قىء.منتهى لأى كلم


In [181]:
df_val.shape[0]+df_test.shape[0]+df_train.shape[0]

16448

In [182]:
vect = CountVectorizer(ngram_range=(1,2)).fit(df_train["text"])
train_df_vectorized = vect.transform(df_train["text"])

In [183]:
clfrNB = MultinomialNB(alpha = 0.1)
clfrNB.fit(train_df_vectorized, df_train.label)

MultinomialNB(alpha=0.1, class_prior=None, fit_prior=True)

In [184]:
from sklearn.model_selection import cross_val_score
cv = cross_val_score(clfrNB,train_df_vectorized,df_train.label,cv=10)

In [185]:
cv

array([0.78036437, 0.76214575, 0.78014184, 0.74468085, 0.76291793,
       0.74062817, 0.75683891, 0.75785208, 0.77001013, 0.75379939])

In [186]:
cv.mean()

0.7609379422369345

In [187]:
vect = TfidfVectorizer(ngram_range=(1,3),min_df=2,max_df=0.5).fit(df_train["text"])
train_df_vectorized = vect.fit_transform(df_train["text"])
clfrNB = MultinomialNB(alpha = 0.1)
clfrNB.fit(train_df_vectorized, df_train.label)

MultinomialNB(alpha=0.1, class_prior=None, fit_prior=True)

In [188]:
cv = cross_val_score(clfrNB,train_df_vectorized,df_train.label,cv=10)

In [189]:
cv

array([0.798583  , 0.79048583, 0.81458967, 0.77507599, 0.80040527,
       0.78014184, 0.79331307, 0.79027356, 0.79432624, 0.77203647])

In [190]:
cv.mean()

0.7909230933307081

In [191]:
# clfrRf = RandomForestClassifier(n_estimators=100,random_state=1)
# clfrRf.fit(train_df_vectorized, df_train.label)
# cv = cross_val_score(clfrRf,train_df_vectorized,df_train.label,cv=10)
# cv

In [192]:
# cv.mean()

In [193]:
accuracy_score(df_val.label,clfrNB.predict(vect.transform(df_val.text)))

0.8017031630170316

In [194]:
import nltk
from nltk.stem.isri import ISRIStemmer
st = ISRIStemmer()
w= 'المسلمون'
print(st.stem(w))

سلم
